#Libraries Importation and Installation

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets, svm
from sklearn.model_selection import train_test_split
import spacy
from spacy.lang.en import English
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from collections import Counter 
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import re
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import keras


RSEED = 42

Using TensorFlow backend.


In [0]:
#!pip install spacy
#!apt install -qq enchant
#!pip install pyenchant
#!python -m spacy download en

#Cleaning

In [0]:
df = pd.read_csv("https://raw.githubusercontent.com/XaviJunior/SBB/master/project_2/Data/train.csv", encoding="utf-8")
df_test = pd.read_csv("https://raw.githubusercontent.com/XaviJunior/SBB/master/project_2/Data/test.csv", encoding="utf-8")
def clean(tweet):
    tweet = re.sub(r" \w{1,3}\.{3,3} http\S{0,}", " ", tweet)
    tweet = re.sub(r"http\S{0,}", " ", tweet)

    tweet = re.sub(r"amp", "", tweet)
  
    special="!#$%&'*+,-./;<=>?@[]^_`{|}~"
    for i in special:
      tweet=tweet.replace(i,' ')
    
#replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)

#remove tokens with number only
    tweet=re.sub(r'\b[0-9]+\b\s*', '', tweet)
    tweet=re.sub(r'[^\w]', ' ', tweet)
    return tweet
    
df["text"] = df["text"].apply(clean)
df_test["text"] = df_test["text"].apply(clean)


In [0]:
nlp = spacy.load('en_core_web_sm')
punctuation = string.punctuation
stop_words = spacy.lang.en.stop_words.STOP_WORDS

def tokenizer(tweet):
    tokens = nlp(tweet)
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    tokens = [word for word in tokens if word not in stop_words and word not in punctuation]
  
    return tokens

#vectorizer = TfidfVectorizer(tokenizer=tokenizer, ngram_range=(1,1))
vectorizer = CountVectorizer(tokenizer=tokenizer, ngram_range=(1,1))

In [4]:
X = vectorizer.fit_transform(df["text"].values.tolist())
X_to_pred = vectorizer.transform(df_test["text"].values.tolist())
y = df["target"].values.tolist()
print(vectorizer.get_feature_names())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RSEED)

['00end', '00pm', '02pm', '05th', '06jst', '0l', '0npzp', '1', '10', '100', '1000s', '1008pla', '1008planet', '100bn', '100nd', '1061thetwister', '10k', '10news', '10th', '10w', '10x', '11', '115film', '12', '12hr', '12jst', '12news', '12th', '12u', '12v', '13pm', '14th', '15', '15moferadio', '15p', '15pm', '15th', '16yr', '17months', '17th', '18jst', '18th', '18w', '1960', '1970', '19th', '19yrs', '1a', '1acd4900c1424d1', '1d', '1pack', '1rockstar62', '1st', '1x1', '2', '20', '20000k', '2007he', '203rd', '20k', '20min', '20skyhawkmm20', '20yrs', '21k', '21st', '22days', '22pm', '23', '231a', '2327564d', '233liveonline', '23rd', '24v', '2500fps', '250k', '260th', '263789f4', '263chat', '265v', '27w', '28pm', '295ss', '29pm', '2a', '2b', '2d', '2day', '2fast2furious', '2for1lapdances', '2hours', '2k13', '2k15', '2leezy', '2minutemix', '2nd', '2pack', '2pcs', '2slow2report', '3', '30', '300', '300k', '300w', '301dc', '30a', '30bst', '30min', '30p', '30pcs', '30pm', '30stm', '30th', '31pm

In [5]:
from sklearn import svm
clf_bern=BernoulliNB(fit_prior=False)
LR = LogisticRegressionCV(solver="lbfgs", max_iter=1000,random_state=RSEED)
clf = svm.NuSVC(gamma='scale',nu=0.48,kernel='linear',probability=True)

eclf1 = VotingClassifier(estimators=[('lr',LR),('bern', clf_bern), ('svm', clf)], voting='soft')
eclf1 = eclf1.fit(X_train, y_train)
print(eclf1.score(X_test,y_test))

0.8076165462902167


In [63]:
from sklearn import svm
clf_bern=BernoulliNB(fit_prior=False)
LR = LogisticRegressionCV(solver="lbfgs", max_iter=1000, random_state=RSEED)
clf = svm.NuSVC(gamma='scale',nu=0.48,kernel='linear',probability=True)
svm=svm.SVC(random_state=RSEED, kernel='rbf')
Lr = LogisticRegression(solver="lbfgs", max_iter=1000, random_state=RSEED)
eclf2 = VotingClassifier(estimators=[('svc',svm),('lrCV', LR), ('bern', clf_bern)], voting='hard')
eclf2 = eclf2.fit(X_train, y_train)
print(eclf2.score(X_test,y_test))

0.8135259356533159


In [0]:
eclf2.fit(X,y)
to_predict = vectorizer.transform(df_test["text"].values.tolist())
df_test["target"] = eclf2.predict(to_predict)  # choose appropriate model

df_test[["id", "target"]].to_csv("UNIL_SBB_Ensemble.csv", index=False)


#Function


In [0]:
def data_augmentation(clf,X,y,threshold):
  df = pd.read_csv("https://raw.githubusercontent.com/XaviJunior/SBB/master/project_2/Data/train.csv", encoding="utf-8")
  df_test["text"] = df_test["text"].apply(clean)
  to_predict = vectorizer.transform(df_test["text"].values.tolist())
  clf.fit(X,y)
  df_test["target"] = clf.predict(to_predict)

  ID=[]
  prob=[]
  Count=0
  for i in eclf1.predict_proba(to_predict):#X_to_pred
    if i[0]> threshold or i[0]<(1-threshold):  #0.7 0.3
      ID.append(Count)
      prob.append(i)
    Count+=1
  
  augment=pd.DataFrame(columns=['id', 'keyword', 'location', 'text', 'target'])
  a=0
  for i in ID:
    augment.loc[a]=df_test.iloc[i,:]
    a+=1

  df_augmented=df.append(augment)
  df_augmented.drop_duplicates(subset ="text", 
                     keep = False, inplace = True)

  df_augmented["text"] = df_augmented["text"].apply(clean)
  X_augmented = vectorizer.fit_transform(df_augmented["text"].values.tolist())
  y_augmented = df_augmented["target"].values.tolist()

  return X_augmented, y_augmented

In [0]:
X_augmented, y_augmented = data_augmentation(eclf1,X,y,0.7)

In [0]:
X_augmented_2, y_augmented_2=data_augmentation(eclf1,X_augmented, y_augmented,0.9)

In [0]:
eclf1.fit(X_augmented_3, y_augmented_3)
df_test = pd.read_csv("https://raw.githubusercontent.com/XaviJunior/SBB/master/project_2/Data/test.csv", encoding="utf-8")
df_test["text"] = df_test["text"].apply(clean)
to_predict = vectorizer.transform(df_test["text"].values.tolist())
df_test["target"] = eclf1.predict(to_predict)  # choose appropriate model

df_test[["id", "target"]].to_csv("UNIL_SBB_Ensemble_Augm.csv", index=False)